# Data Extraction from Meeting Minute PDFs

PDFs we are extracting from are UCSB AS F&B Meeting Minutes, publicly available at [AS F&B Committee Minutes](https://asfb.as.ucsb.edu/minutes2018-2019/) -- We are using Fall 2024 and Winter 2025 quarter minutes.

In [10]:
import pdfplumber
import re
import pandas as pd
import logging

In [24]:
# Ignore non-critical warnings from pdfminer through pdfplumber
logging.getLogger("pdfminer").setLevel(logging.ERROR)

# Folder of pdfs, from UCSB AS F&B Meeting Minutes, publicly available, see above
pdf_folder = "meeting-mins-pdfs/"

# This function will convert pdf pages holding relevant motions into text, then return it in all in one big string
def motions_text_from_pdf(pdf_path):

    collecting = False
    out = ''
    
    with pdfplumber.open(pdf_path) as pdf:
         
        for page in pdf.pages: 
            
            text = page.extract_text()

            if not collecting:

                if "action items" in text.lower():
                    collecting = True
                    
            if collecting:
                out += text
                
        return out

# Looks for and returns list of only 
def find_motions(text):

    pattern = r"motion language(.*?)action: passed"
    motions = re.findall(pattern, text.lower(), flags=re.DOTALL)
    return motions
    

print(find_motions(motions_text_from_pdf(r'meeting-mins-pdfs/10.07.2024 Finance Committee Meeting Minutes.pdf')))

Additionalapprovalrequired:YESSenate
E. Acceptance of Action Summary
F. Action Items
C-1. New Business
1. Debate Team at UCSB requesting for $5,000.00
a. Jannese Davidson Memorial tournament - $1,200.00
b. Sunset Cliffs Classic - $1,200.00
c. Dahlin Invitational - $400.00
d. Griffin Invitational - $1,000.00
e. PSCFA Fall Champs - $1,200.00
Singh: That seems okay.
MOTION/SECOND: Wang/Mazarei
Motion Language: Motion to fully fund Debate Team at UCSB $5,000 from Academic
Teams
ACTION: Passed at 5:50PM
Advisor/Staff Instruction/Request:
Responsible for Follow-through:
Additionalapprovalrequired:YESSenate
2. Untitled Dance Company requesting for $1,225.00
a. Robertson Gymnasium Rental - $1,225.00
Singh: I recommend this comes out of SEAL Fall.
MOTION/SECOND: Sim/Budwal
Motion Language: Motion to fully fund United Dance Company $1,225 from SEAL Fall
Quarter Fund.ACTION: Passed at 5:52PM
Advisor/Staff Instruction/Request:
Responsible for Follow-through:
Additionalapprovalrequired:YESSenate
3.